<a href="https://colab.research.google.com/github/deangarcia/NLP/blob/main/NLP_Assignment2_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment, you will familiarize yourself with:

    spaCy
    numpy
    PyTorch

to develop

    Multiple sentiment analysis models

to be able to

    Predict whether a given review is positive or negative

Before we begin, make sure your runtime is set to GPU -- 

Runtime > Change runtime type set to GPU

First we will load in some data.

Provided is code that will download a file and rename it to reviews.tar.gz


In [ ]:

!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 
!tar -xf aclImdb_v1.tar.gz
positive_reviews = !ls -1 -d aclImdb/train/pos/*
negative_reviews = !ls -1 -d aclImdb/train/neg/*
unsupervised_reviews = !ls -1 -d aclImdb/train/unsup/*

vocabulary = set()

with open('./aclImdb/imdb.vocab') as vocab_file:
  for word in vocab_file:
    vocabulary.add(word.rstrip())

with open(positive_reviews[0]) as review:
  for line in review:
    for word in line.rstrip().split(' '):
      if word not in vocabulary:
        print(word)


--2022-03-01 19:55:27--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  17.3MB/s    in 8.2s    

2022-03-01 19:55:36 (9.77 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!tar -xf aclImdb_v1.tar.gz
positive_reviews = !ls -1 -d aclImdb/train/pos/*
negative_reviews = !ls -1 -d aclImdb/train/neg/*
unsupervised_reviews = !ls -1 -d aclImdb/train/unsup/*

vocabulary = set()

with open('./aclImdb/imdb.vocab') as vocab_file:
  for word in vocab_file:
    vocabulary.add(word.rstrip())

with open(positive_reviews[0]) as review:
  for line in review:
    for word in line.rstrip().split(' '):
      if word not in vocabulary:
        print(word)

Included is a vocabulary of all the words we can potentially see -- No '\<UNK\>' *here*

In [ ]:
!head aclImdb/imdb.vocab

the
and
a
of
to
is
it
in
i
this


In [ ]:
positive_reviews = !ls -1 -d aclImdb/train/pos/*
negative_reviews = !ls -1 -d aclImdb/train/neg/*
unsupervised_reviews = !ls -1 -d aclImdb/train/unsup/*

vocabulary = set()

with open('./aclImdb/imdb.vocab') as vocab_file:
  for word in vocab_file:
    vocabulary.add(word.rstrip())

with open(positive_reviews[0]) as review:
  for line in review:
    for word in line.rstrip().split(' '):
      if word not in vocabulary:
        print(word)

In [ ]:
print('Positive #',len(positive_reviews))
print('Negative #',len(negative_reviews))
print('Unsupervised #',len(unsupervised_reviews))

Positive # 12500
Negative # 12500
Unsupervised # 50000


So, we have 12500 reviews for positive and negative classifications, and 50000 that are unlabeled.  

In [ ]:
vocabulary = set()

with open('./aclImdb/imdb.vocab') as vocab_file:
  for word in vocab_file:
    vocabulary.add(word.rstrip())

with open(positive_reviews[0]) as review:
  for line in review:
    for word in line.rstrip().split(' '):
      if word not in vocabulary:
        print(word)

In [ ]:
with open(positive_reviews[0]) as review:
  for line in review:
    for word in line.rstrip().split(' '):
      if word not in vocabulary:
        print(word)
        

Bromwell
High
comedy.
It
life,
"Teachers".
My
35
Bromwell
High's
"Teachers".
The
financially,
teachers'
pomp,
situation,
I
students.
When
I
school,
I
.........
..........
High.
A
line:
INSPECTOR:
I'm
teachers.
STUDENT:
Welcome
Bromwell
High.
I
Bromwell
High
fetched.
What
isn't!


Looking at the words in a review that don't show up in the vocab, we need to handle punctuation in a more delicate way (also, we will be making everything lower cased).  For this, we will turn to a standard modern NLP library spaCy.  spaCy is a library that handles a number of different low-level NLP tasks like tokenization, part-of-speech recognition, and named entity recognition.  For now, we will be focusing on the tokenization aspect.



In [ ]:
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)
counts = {}
for review in positive_reviews + negative_reviews:
  with open(review) as review:
    for line in review:
      for word in tokenizer(line.rstrip().lower()):
        if word.text not in vocabulary:
          pass
        else:
          counts[word.text] = counts.get(word.text,0) + 1


In [ ]:
count2words = {}
for word in counts:
  count = counts[word]
  if count not in count2words:
    count2words[count] = []
  count2words[count].append(word)



In [ ]:
running_total = 0
vocabulary = []
for count in reversed(sorted(count2words)):
  running_total += len(count2words[count])
  vocabulary += count2words[count]
  if running_total > 10000:
    break

In [ ]:
print(vocabulary)

['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'you', "n't", 'not', 'are', 'he', 'his', 'have', 'be', 'one', 'all', 'at', 'they', 'by', '!', 'an', 'who', 'from', 'like', 'so', 'her', 'or', 'just', 'there', 'do', 'about', 'has', 'out', 'if', 'what', 'some', 'good', '?', 'she', 'more', 'very', 'when', 'would', 'up', 'no', 'even', 'time', 'can', 'my', 'which', 'only', 'story', 'really', 'had', 'their', 'see', 'were', 'we', 'me', 'did', 'does', 'well', 'than', 'much', 'could', 'get', 'been', 'will', 'people', 'bad', 'into', 'other', 'because', 'great', 'also', 'first', 'how', 'most', 'him', 'made', 'its', 'make', 'then', 'way', 'them', 'too', 'any', 'movies', 'after', 'br', 'think', 'characters', 'character', 'two', 'watch', 'films', 'many', 'seen', 'being', 'never', 'plot', 'life', 'little', 'love', 'acting', 'where', 'best', 'over', 'show', 'know', 'off', 'ever', 'man', 'your', 'better', 'still', 'here', 'e

In [9]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 
!tar -xf aclImdb_v1.tar.gz
positive_reviews = !ls -1 -d aclImdb/train/pos/*
negative_reviews = !ls -1 -d aclImdb/train/neg/*
unsupervised_reviews = !ls -1 -d aclImdb/train/unsup/*

vocabulary = set()

with open('./aclImdb/imdb.vocab') as vocab_file:
  for word in vocab_file:
    vocabulary.add(word.rstrip())

with open(positive_reviews[0]) as review:
  for line in review:
    for word in line.rstrip().split(' '):
      if word not in vocabulary:
        print(word)
        
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.Defaults.create_tokenizer(nlp)
counts = {}
for review in positive_reviews + negative_reviews:
  with open(review) as review:
    for line in review:
      for word in tokenizer(line.rstrip().lower()):
        if word.text not in vocabulary:
          pass
        else:
          counts[word.text] = counts.get(word.text,0) + 1

count2words = {}
for word in counts:
  count = counts[word]
  if count not in count2words:
    count2words[count] = []
  count2words[count].append(word)

running_total = 0
vocabulary = []
for count in reversed(sorted(count2words)):
  running_total += len(count2words[count])
  vocabulary += count2words[count]
  if running_total > 10000:
    break

--2022-03-08 22:56:25--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz.1’

aclImdb_v1.tar.gz.1 100%[===================>]  80.23M  32.9MB/s    in 2.4s    

2022-03-08 22:56:28 (32.9 MB/s) - ‘aclImdb_v1.tar.gz.1’ saved [84125825/84125825]

Bromwell
High
comedy.
It
life,
"Teachers".
My
35
Bromwell
High's
"Teachers".
The
financially,
teachers'
pomp,
situation,
I
students.
When
I
school,
I
.........
..........
High.
A
line:
INSPECTOR:
I'm
teachers.
STUDENT:
Welcome
Bromwell
High.
I
Bromwell
High
fetched.
What
isn't!


In [3]:
print(counts)

{'bromwell': 8, 'high': 2145, 'is': 110134, 'a': 161674, 'cartoon': 534, 'comedy': 3169, 'it': 93207, 'ran': 236, 'at': 23256, 'the': 328324, 'same': 4023, 'time': 12315, 'as': 46135, 'some': 15523, 'other': 9013, 'programs': 66, 'about': 17243, 'school': 1633, 'life': 6424, 'such': 5094, 'teachers': 76, 'my': 12036, 'years': 4407, 'in': 92779, 'teaching': 82, 'profession': 62, 'lead': 1288, 'me': 10525, 'to': 135338, 'believe': 2487, 'that': 73033, 'satire': 255, 'much': 9612, 'closer': 205, 'reality': 956, 'than': 9914, 'scramble': 6, 'survive': 244, 'financially': 28, 'insightful': 64, 'students': 358, 'who': 21304, 'can': 12170, 'see': 11356, 'right': 3239, 'through': 4905, 'their': 11366, 'pathetic': 448, 'pomp': 9, 'pettiness': 2, 'of': 145587, 'whole': 3058, 'situation': 642, 'all': 23271, 'remind': 157, 'schools': 66, 'i': 82430, 'knew': 894, 'and': 162830, 'when': 13861, 'saw': 3156, 'episode': 1596, 'which': 11959, 'student': 386, 'repeatedly': 119, 'tried': 761, 'burn': 124,

In [ ]:
!wget -O corpus.zip https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1&isAllowed=y 
!unzip -o corpus.zip
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 
!tar -xf aclImdb_v1.tar.gz
files = !ls Shake*/*/*_noSCRB.txt 
files = [file[1:-1] for file in files]

!pip install spacy

We can now tokenize our text and the things we ignore are *mostly* puncutation (most tokenizers will split contractions like I'm into I and 'm which happens here, but the original vocab doesn't expect -- we will just let that happen here)


#Step 1

Fill out the function below to:

* Tokenize a document
* Extract all n-grams and their counts of the given order

* It's ok to utilize your work from the first assignment here (although use SpaCy and go to lower case)

In [11]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)

# Construction 2
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

def get_n_grams(filename,n):
  ngrams = {}
  texts = []
  test = []

  filex = open(filename,'r')
  tokens = tokenizer(filex.read())
  for token in tokens:
    test.append(token.lemma_.lower())
  texts.append(test)
  #Text is a list of lists
  for text in texts:
    # iterate through each list (-n + 1 because of the nested for loop we dont want to go out of range and len(array) doesnt take into account 0)
    for i in range(len(text) - n + 1):
      temp = [] 
      # now iterate through the list j values which is the length of n to save our string values into a tuple
      for j in range(n):
        temp.append(text[i+j])
      tuple_temp = tuple(temp)
      # save the tuple as a key at increment it by one if it exists already 
      if tuple_temp in ngrams.keys():
        ngrams[tuple_temp] += 1
      # or set it to one on the first occurence
      else:
        ngrams[tuple_temp] = 1

  return ngrams


We are going to use Stochastic Gradient Descent to learn the weights for our regression, and we will be utilizing the PyTorch library to do so

In [12]:
import torch
import torch.nn as nn

With the ability to extract out n-grams, we will now be constructing a linear classifier.  To do this, we will need to set up our data.  

#Step 2

* Make a $|D|\times|unigrams|$ tensor (`torch.Tensor`) , `X_unigrams` that contains all of the unigram counts for the documents -- each row should be a document and each column should be a unigram -- each cell corresponding to the number of times the unigram corresponding to the column shows up in the document corresponding to the row  
i.e., $X_{unigrams}[i,j] = $ Count of unigram $j$ in document $i$
* Make a $|D|\times1$ tensor, `Y` that contains the ratings for the documents -- we will say that a positive review has a rating of `1` and a negative review is `0`
*In torch, if we want to use a GPU for the training we need to move the data to the GPU using `.to('cuda')`.  For this to work you will need to make sure you are using a GPU instance (Runtime > Change Runtime Type > Hardware Accelerator = GPU)

In [129]:
n = 1
if torch.cuda.is_available():
  device = torch.device("cuda") 
data = []
ratings = []
count = 0
for filename in positive_reviews + negative_reviews:
  if count > 20:
    break
  else:
    count = count + 1
  substring = "pos"
  rating_val = []
  if substring in filename:
    ratings.append(1)
  else:
    ratings.append(0)
  row_data = []
  gram = get_n_grams(filename, n)
  for i in range(len(vocabulary)):
    temp_list = []
    for j in range(n):
      temp_list.append(vocabulary[i])
    cmp_tuple = tuple(temp_list)
    if cmp_tuple in gram.keys():
      row_data.append(gram[cmp_tuple])
    else:
      row_data.append(0)
  data.append(row_data)

if torch.cuda.is_available():
  device = torch.device("cuda")
  X_unigrams = torch.FloatTensor(data).to(device)
  y = torch.FloatTensor(ratings).to(device)
  y = y.view(y.shape[0], 1)

print(X_unigrams) # not sure if the data model is setup correctly here how would we do a bigram or trigram?
print(y)



tensor([[ 9.,  1.,  4.,  ...,  0.,  0.,  0.],
        [24.,  4., 17.,  ...,  0.,  0.,  0.],
        [10.,  4.,  6.,  ...,  0.,  0.,  0.],
        ...,
        [ 7.,  5.,  3.,  ...,  0.,  0.,  0.],
        [17.,  4.,  0.,  ...,  0.,  0.,  0.],
        [13.,  6.,  5.,  ...,  0.,  0.,  0.]], device='cuda:0')
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0')


We will first construct a module that does what we want.  In torch you implement modules and you define the forward pass of the model (Torch uses autograd to automatically compute the backward pass based on the forward pass). 

#Step 3
* Implement the linear regression using Torch
* You will want to use a `torch.nn.Linear` layer -- The linear layer is essentially a matrix multiplication as shown in class  -- think about what the input dimension and output dimensions should be (how big is our input, how many things are we predicting)


In [90]:
class LinearRegression(nn.Module):
  def __init__(self, size):
    super(LinearRegression, self).__init__()
    self.linear = nn.Linear(size , 1).to(device)

  def forward(self,X):
    out = self.linear(X).to(device)
    return out

With the above model, we now need to actually train it.  We are going to use a pretty simple training regimen here.

#Step 4 
* Create the optimizer -- I recommend using the Adam optimizer (`optim.Adam`) -- optimizers take in the parameters that they are optimizing (`model.parameters`) and other hyperparameters like the learning rate (`lr=learning_rate`)
* Loop for the number of `epochs` supplied to the training code -- this will be the number of passes over our data
* In each epoch we will loop over the data in batches -- go from 0 up to the size of `X` in steps of size `batch_size`
* Within each training step we will need to follow the steps of training a neural network:
 * Zero out the gradients in the optimizer (by default the gradients are kept around) by calling `.zero_grad()` on the optimizer
 * Get the current batch -- we can use *slicing* to get certain elements in our input and output.  E.g. `X[i:i+step_size,:]` would return a Tensor with elements from X from row i to row i+step_size and all of the columns
 * Run the model on the batch making sure to assign the result to a variable -- you can either call `.forward(...)` or more simply just call the model `(...)`
 * Calculate the loss of the output -- this is done by calling the `loss_criterion` with the predicted values and the true values as the first and second arguments respectively
 * Run the loss in the backward direction -- Call `.backward()` on the loss calcualted in the previous step
 * Step the optimizer -- Call `.step()` on the optimizer
 * You might want to do something like log the value of the loss -- this can be gotten by calling `.item()` on the loss calculated above -- perhaps do this every 5, 10, 50, 100, 500 epochs (your choice)

In [136]:
import torch.optim as optim


def train(X, Y, model, batch_size, epochs, learning_rate, loss_criterion):
  """Train using a pytorch model with batches"""
  if torch.cuda.is_available():
    device = torch.device("cuda")
  
  print(model.parameters())
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  for i in range(epochs):
    for step_size in range(batch_size):
      optimizer.zero_grad()
      output = model(X[i:i+step_size,:])
      loss = loss_criterion(output, Y[i:i+step_size,:])
      loss.backward()
      optimizer.step()


#Step 5
* Construct your model 
* Just as with the data above, if you want to use a GPU we have to tell torch to move the model (and parameters to the gpu) so you need to call `.to('cuda')` (Note this does not change the original model, it returns a model on the gpu so you probably want to do something like `model = model.to('cuda')`
* Call the above training code -- watch how the loss changes as the model trains -- experiment with different training hyperparameters -- learning rate and epoch -- see when the model hits a minima
* For a linear regression, we are using Mean Square Error as our loss `torch.nn.MSELoss`

In [159]:
if torch.cuda.is_available():
  device = torch.device("cuda")


linear_model = LinearRegression(len(vocabulary)).to(device)

epochs = 5
batch_size = 10
learning_rate = 0.01
loss_criterion = nn.MSELoss()
train(X_unigrams, y, linear_model, batch_size, epochs, learning_rate, loss_criterion)

<generator object Module.parameters at 0x7f748bde1a50>



#Step 6
* Implement the class below -- you should have two linear weights  (Input -> Hidden), (Hidden -> Output)
* In your forward, make sure you apply a nonlinear activation function after going from (Input -> Hidden) before (Hidden -> Output) -- you can choose the nonlinear activation (sigmoid, tanh, relu, leaky-relu are all fine)
* Train the model

###Question 2
How does this model compare to the above model?

In [160]:
class NonLinearRegression(nn.Module):
  def __init__(self, size, hidden_size):
    super(NonLinearRegression, self).__init__()
    self.non_linear = nn.Linear(size , hidden_size).to(device)
    self.relu = nn.ReLU()
    self.non_linear_2 = nn.Linear(hidden_size , 1).to(device)
    self.sigmoid = nn.Sigmoid()

  def forward(self,X):
    out = self.non_linear(X).to(device)
    out = self.relu(out).to(device)
    out = self.non_linear_2(out).to(device)
    out = self.sigmoid(out).to(device)
    return out

In [161]:
if torch.cuda.is_available():
  device = torch.device("cuda")

hidden_size = ((len(vocabulary) * 2) / 3) + 1
hidden_size = int(hidden_size)
nonlinear_model = NonLinearRegression(len(vocabulary), hidden_size).to(device)

epochs = 5
batch_size = 10 # what is the purpose of the batches?
learning_rate = 0.01 # what does this variable do?
loss_criterion = nn.MSELoss()
train(X_unigrams, y, nonlinear_model, batch_size, epochs, learning_rate, loss_criterion)

<generator object Module.parameters at 0x7f7404576750>


Ok, we have a linear regression and a non-linear regression -- let's do a Logistic Regression and non-linear Logistic regression now:
#Step 7
* Implement a standard logistic regression and a logistic regression with a hidden layer and non-linear activation
* These should be very similar to above except they should have `torch.Sigmoid` applied to the output
* Note, `MSELoss` is no longer applicable here -- we need to use Binary Cross Entropy Loss `torch.nn.BCELoss`

In [168]:
class LogisticRegression(nn.Module):
  def __init__(self, size):
    super(LogisticRegression, self).__init__()
    self.logistic = nn.Linear(size, 1)

  def forward(self,X):
    out = torch.sigmoid(self.logistic(X))
    return out


class LogisticANN(nn.Module):
  def __init__(self, size, hidden_size):
    super(LogisticANN, self).__init__()
    self.non_linear = nn.Linear(size , hidden_size).to(device)
    self.relu = nn.ReLU()
    self.non_linear_2 = nn.Linear(hidden_size , 1).to(device)
    self.sigmoid = nn.Sigmoid()

  def forward(self,X):
    out = self.non_linear(X).to(device)
    out = self.relu(out).to(device)
    out = self.non_linear_2(out).to(device)
    out = self.sigmoid(out).to(device)
    return out

In [169]:
if torch.cuda.is_available():
  device = torch.device("cuda")

logistic_reg_model = LogisticRegression(len(vocabulary)).to(device)

epochs = 5
batch_size = 10 # what is the purpose of the batches?
learning_rate = 0.01 # what does this variable do?
loss_criterion = nn.BCELoss()
train(X_unigrams, y, logistic_reg_model, batch_size, epochs, learning_rate, loss_criterion)

hidden_size = ((len(vocabulary) * 2) / 3) + 1
hidden_size = int(hidden_size)
logisticann_model = LogisticANN(len(vocabulary), hidden_size).to(device)

epochs = 5
batch_size = 10 # what is the purpose of the batches?
learning_rate = 0.01 # what does this variable do?
loss_criterion = nn.BCELoss()
train(X_unigrams, y, logisticann_model, batch_size, epochs, learning_rate, loss_criterion)

<generator object Module.parameters at 0x7f7404aab750>
<generator object Module.parameters at 0x7f7404aab650>


# Step 8
* This step is somewhat freeform -- just as it would be if you were doing exploratory resarch
* Try doing some exploration of different models.  Some things you might try are:
 * different configurations of inputs (different order n-grams, combinations of n-grams)
 * different numbers and sizes of input layers
 * different non-linear activations

* Normally, you should pick the model that does the best not on training data, but on the evaluation data.  Since this dataset doesn't come with an evaluation set, we will instead be using the test set as our evaluation set (this is obviously poor practice, and should not be done in the real world).  Or rather we will be using a portion of the test files as evaluation.

* Pick a model architecture, train it, run it on the eval data 
* Do this multiple times, keep the best model 

#### Question 3:
How do you pick the "best" model -- what metric do you want to use here?

* As a note, there will be n-grams that show up in the evaluation and training sets that don't show up in the training data, you will have to be able to account for this (of course, ignoring is acceptable) but your code shouldn't fail


In [ ]:
positive_raw_test = !ls -1 -d aclImdb/test/pos/*
negative_raw_test = !ls -1 -d aclImdb/test/neg/*

In [ ]:
positive_eval = positive_raw_test[:len(positive_raw_test)//4]
negative_eval = negative_raw_test[:len(negative_raw_test)//4]
positive_test = positive_raw_test[len(positive_test)//4:]
negative_test = negative_raw_test[len(negative_raw_test)//4:]

In [ ]:
raise NotImplementedError("STEP 7 Search through more advanced models")

## Bonus: Augment with unsupervised data
* The dataset comes with more unsupervised data than supervised -- come up with a way to augment your training set with the unsupervised data, such that your model performance improves (to do this comparison keep model architecture the same, and compare the augmented data model vs the original on the evaluation set)